In [1]:
# load all packages in Julia
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [3]:
#open csv files to define sets, lines, gens and and loads/demand
datadir = joinpath("ReducedNetwork") 
gens = CSV.read(joinpath(datadir,"GenRedNet2.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"TransRedNet.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"demandRedNet.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids #changed to read columns rather than rows
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines.fromnode
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

#lines.capacity = 0.4*lines.capacity
# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
#loads = loads[:,["connnode","interval-1_load"]]
#rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.0122,0.02,1,10000,1,50.0
2,1,3,0.007,0.15,1,10000,2,6.66667
3,1,2,0.0122,0.02,1,10000,3,50.0
4,1,3,0.007,0.15,1,10000,4,6.66667
5,2,4,0.0004,0.065,1,10000,5,15.3846
6,2,4,0.0004,0.065,1,10000,6,15.3846
7,4,7,0.00211,0.0135,1,10000,7,74.0741
8,4,6,0.0013,0.023,1,10000,8,43.4783
9,4,6,0.0013,0.023,1,10000,9,43.4783


In [4]:
#define optimization function 
#NationalPricing attempt
#Function to solve DC F problem using IEEE test cases
#Inputs:
    #gen_info -- dataframe with generator info
    #line_info -- dataframe with transmission lines info
    #loads  -- dataframe with load info
#
function np_attempt(gens, lines, loads)
    NP = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
    
      # Set of all nodes
    N = sort(union((lines.fromnode), 
            (lines.tonode))) #took out unique
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system *LOOK INTO THIS FURTHER*
    
    # Decision variables   
    @variables(NP, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows along each line
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    #fix(THETA[1],0)
                
    # Objective function
    @objective(NP, Min, 
        sum(gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(NP, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(NP, cMaxGen[g in G],
                   GEN[g] <= gens[g,:pgmax])
    
 
    


    # Solve statement (! indicates runs in place)
    optimize!(NP)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))

    #load payment
    #loadpay = sum(dual.(cBalance)*(loads for loads in loads[loads.connnode, :demand]))
    

    #congestion cost
    
    
    #Total Gen prof
    #genprof = sum(GEN[g] for g in gens[gens.connnode .== i,:id]) * sum()
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        #loadpay,
        cost = objective_value(NP),
        status = termination_status(NP)
    )
end

np_attempt (generic function with 1 method)

In [5]:
#solve 
sol = np_attempt(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 66 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-95); columns 0(-936); elements 0(-232) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00


In [6]:
#generation output
sol.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,2,0.0
6,2,0.0
7,2,0.0
8,3,0.0
9,3,0.0


In [7]:
#prices output
sol.prices

Row,node,value
,Int64,Float64
1,1,-0.0
2,2,-0.0
3,3,-0.0
4,4,-0.0
5,5,-0.0
6,6,-0.0
7,7,-0.0
8,8,-0.0
9,9,-0.0


In [8]:
#flows output
sol.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,0.0
2,1,3,0.0
3,1,2,0.0
4,1,3,0.0
5,2,4,0.0
6,2,4,0.0
7,4,7,0.0
8,4,6,0.0
9,4,6,0.0


In [9]:
#sum of gens
sum(sol.generation[:,2])

LoadError: UndefVarError: `gendf` not defined

In [ ]:
#sum of prices per node
sum(sol.prices[:,2])

In [ ]:
#sum of flows per node
sum(flowsdf[:,3])